<a href="https://colab.research.google.com/github/abdullahmashhadi/chatWithMultiplePDFs/blob/main/Chat_PDF_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Chat with Multiple PDF's Llama 2 + Pinecone + LangChain**

#**Step: 01: Install All the Required Packages**

In [1]:
!pip install langchain
!pip install pinecone-client
!pip install sentence_transformers
!pip install pdf2image
!pip install pypdf
!pip install xformers
!pip install bitsandbytes accelerate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 9.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-many

#**Step 2: Import All the Required Libraries**

In [2]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.9 MB/s eta 0:00:00


In [3]:
!pip install --upgrade fastai torchaudio

INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.6 MB/s eta 0:00:00
Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 5.4 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.0.0
    Uninstalling triton-3.0.0:
      Successfully uninstalled triton-3.0.0
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 9.1.0.70
    Uninstalling nvidia-cudnn-cu12-9.1.0.70:
      Successfully uninstalled nvidia-cudnn-cu12-9.1.0.70
  Attempting uninstall: torch
    Found existing installation: torch 2.4.0
    Uninstalling torch-2.4.0:
      Successfully uninstalled torch-2.4.0
ERROR: pip's de

In [4]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import os
import sys

In [5]:
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA

#**Step 3: Load the PDF Files**

In [6]:
!mkdir pdfs

In [7]:
# !gdown 1hPQlXrX8FbaYaLypxTmeVOFNitbBMlEE -O pdfs/yolov7paper.pdf
# !gdown 1vILwiv6nS2wI3chxNabMgry3qnV67TxM -O pdfs/rachelgreecv.pdf
!gdown 1fyldzdgRWs3CRkix_UG-rj2I_J0_a1p8 -O pdfs/7habits.pdf
!gdown 1_yXQJ6M3Ql07t5lA1v1vgflXHOL9eLkr -O pdfs/goodToGreat.pdf

Downloading...
From: https://drive.google.com/uc?id=1fyldzdgRWs3CRkix_UG-rj2I_J0_a1p8
To: /content/pdfs/7habits.pdf
100% 1.44M/1.44M [00:00<00:00, 58.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_yXQJ6M3Ql07t5lA1v1vgflXHOL9eLkr
To: /content/pdfs/goodToGreat.pdf
100% 12.5M/12.5M [00:00<00:00, 106MB/s] 


#**Step 04: Extract the text from the PDF's**

In [8]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [9]:
print(data)

[Document(metadata={'source': 'pdfs/7habits.pdf', 'page': 0}, page_content='T H E S E V E N H A B I T S O F H I G H L Y E F F E C T I V E P E O P L E B r o u g h t t o y o u b y F l y H e a r t \nTHE SEVEN HABITS OF HIGHLY EFFECTIVE PEOPLE \nStephen R. Covey \n'), Document(metadata={'source': 'pdfs/7habits.pdf', 'page': 1}, page_content="T H E S E V E N H A B I T S O F H I G H L Y E F F E C T I V E P E O P L E B r o u g h t t o y o u b y F l y H e a r t \nStephen Covey has written a remark able book about the human condition, so elegantly written, so \nunderstanding of our embedded concer ns, so useful for our organizatio n and personal lives, that it's \ngoing to be my gift to everyone I know. \n-- Warren Bennis, author of On Becoming a Leader \nI've never known any teacher or mentor on im proving personal effectiveness to generate such an \noverwhelmingly positive reaction.... This book captures beautifull y Stephen's philosoph y of principles. \nI think anyone reading it will quickl

#**Step 05: Split the Extracted Data into Text Chunks**

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [11]:
docs = text_splitter.split_documents(data)

In [12]:
print(len(docs))

2888


In [13]:
docs[3]

Document(metadata={'source': 'pdfs/7habits.pdf', 'page': 1}, page_content="family, relationships, communicating. \n-- Brian Tracy, author of Psychology of Achievement \nStephen R. Covey's book teaches with power, co nviction, and feeling. Both the content and the \nmethodology of these principles form a solid foundati on for effective communication. As an educator, \nI think this book to be a significant addition to my library. \n-- William Rolfe Kerr, Utah Commissioner of Higher Education")

In [14]:
docs[4]

Document(metadata={'source': 'pdfs/7habits.pdf', 'page': 1}, page_content='Few students of management and organization -- and people -- have thought as long and hard about \nfirst principles as Stephen Covey. In The Seven Ha bits of Highly Effective People, he offers us an \nopportunity, not a how-to guide. The opportunity is to explore ourselves and our impact on others, \nand to do so by taking advantage of his profound insights. It is a wonderful book that could change \nyour life. \n-- Tom Peters, author of In Search of Excellence')

#**Step 06: Downlaod the Embeddings From the Hugging Face**

In [15]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
query_result = embeddings.embed_query("Hello World")

In [17]:
print("Length", len(query_result))

Length 384


#**Step 07: Initializing the Pinecone**

In [18]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '732c62e3-4f9c-47fb-ae99-8b1d1e636fe7')
# PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'us-east-1-aws')

In [23]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '732c62e3-4f9c-47fb-ae99-8b1d1e636fe7')
# PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'us-east-1-aws')

# Create a Pinecone instance using the API key
pinecone_instance = pinecone.Pinecone(api_key=PINECONE_API_KEY)

index_name="langchainpinecone"

#**Step 08: Create Embeddings for each of the Text Chunk**

In [29]:
texts = list(map(lambda x: x.page_content.replace("\n", " "), docs))

In [31]:
import os
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings
# Here we are setting the API key as an environment variable
os.environ["PINECONE_API_KEY"] = "732c62e3-4f9c-47fb-ae99-8b1d1e636fe7"

# Now you can use the from_texts method which will internally access the key
pinecone_index = Pinecone.from_texts(texts, embeddings, index_name=index_name)

#If you already have an index, you can load it like this

In [34]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)


#**Step 09: Similarity Search**

In [32]:
query = "What are the 7 habits of highly effective people?"

In [35]:
docs = docsearch.similarity_search(query, k=4)

In [36]:
docs

[Document(page_content='T H E S E V E N H A B I T S O F H I G H L Y E F F E C T I V E P E O P L E B r o u g h t t o y o u b y F l y H e a r t  THE SEVEN HABITS OF HIGHLY EFFECTIVE PEOPLE  Stephen R. Covey'),
 Document(page_content='the Seven Habits, and will be used throughout this b ook as we explore both the sequential relationship  between the habits and also their synergy -- how, in relating to each other, th ey create bold new forms  of each other that add even more to their value. Each concept or habit will be highlighted as it is  introduced.  Effectiveness Defined  The Seven Habits are habits of effectiveness. Because they are based on principles, they bring the'),
 Document(page_content='T H E S E V E N H A B I T S O F H I G H L Y E F F E C T I V E P E O P L E B r o u g h t t o y o u b y F l y H e a r t  The Seven Habits of Highly Effective People su ggests a discipline for our personal dealings with  people which would be undoubtedly valuable  if people stopped to think about

#**Step 10: Creating a Llama2 Model Wrapper**

In [37]:
from huggingface_hub import notebook_login
import torch

In [38]:
notebook_login()


In [40]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [41]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             load_in_8bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [42]:

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [43]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


#**Step 11: Create a Prompt Template**

In [44]:
SYSTEM_PROMPT = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer."""


In [45]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

In [46]:
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT + E_SYS

In [47]:
instruction = """
{context}

Question: {question}
"""

In [48]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [49]:
template

"[INST]<<SYS>>\nUse the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n<</SYS>>\n\n\n{context}\n\nQuestion: {question}\n[/INST]"

In [50]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [51]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [52]:
result = qa_chain("How a company can become successful in the World?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [53]:
result['result']

'[INST]<<SYS>>\nUse the following pieces of context to answer the question at the end.\nIf you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n<</SYS>>\n\n\nstudied.\tIt\tis\tabout\t the\tquestion –Can\ta\tgood\tcompany\tbecome\ta\tgreat\tcompany and,\tif\tso,\thow?—and\tour\tsearch\tfor\ttimeless,\tuniversal\tanswers\tthat\tcan\tbe applied\tby\tany\torganization. Our\tfive-year\tquest\tyielded\tmany\tinsights,\ta\tnumber\tof\tthem\tsurprising\tand quite\tcontrary\tto\tconventional\twisdom,\tbut\tone\tgiant\tconclusion\tstands above\tthe\tothers:\tWe\tbelieve\tthat\talmost\t any \torganization\tcan\tsubstantially improve\tits\tstature\tand\tperformance,\tperhaps\teven\tbecome\tgreat,\tif\tit\n\nmore successful\tthan\tthe\tgood-to-great\tcompany—larger\tand\tmore\tprofitable,\twith\ta\n\nQuestion: How a company can become successful in the World?\n[/INST]  Based on the context provided, the answer to the question "How a company can become success

In [55]:
while True:
    user_input=input(f"prompt:")
    if user_input=='exit':
        print('Exiting')
        sys.exit()
    if user_input=='':
        continue
    result=qa_chain({'query':user_input})
    print(f"Answer:{result['result']}")

prompt:What are the 7 habits of highly effective people?
Answer:[INST]<<SYS>>
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<</SYS>>


T H E S E V E N H A B I T S O F H I G H L Y E F F E C T I V E P E O P L E B r o u g h t t o y o u b y F l y H e a r t  THE SEVEN HABITS OF HIGHLY EFFECTIVE PEOPLE  Stephen R. Covey

the Seven Habits, and will be used throughout this b ook as we explore both the sequential relationship  between the habits and also their synergy -- how, in relating to each other, th ey create bold new forms  of each other that add even more to their value. Each concept or habit will be highlighted as it is  introduced.  Effectiveness Defined  The Seven Habits are habits of effectiveness. Because they are based on principles, they bring the

Question: What are the 7 habits of highly effective people?
[/INST]  Based on the text provided, the 7 habits of highl

SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
